In [23]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import copy
import ast
import attributes

from ipywidgets import interact, Dropdown
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

pd.set_option('display.max_rows', 200)

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

AIRFLOW_WASSTRAAT_CONFIG = "./wasstraat_config/Wasstraat_Config_Harmonize.xlsx"
SUGGESTIE_XL = "./wasstraat_config/Wasstraat_Attribuut_Suggestie.xlsx"

In [3]:
xl = pd.read_excel(AIRFLOW_WASSTRAAT_CONFIG, None);
df_table = xl['Objecten']
df_table.head(30)

,Unnamed: 0,Object,Tabellen,Overerven,Samenvoegen,ABR-code
0,0,Ignore,"["".*backup.*"", "".*kopie.*""]",NaN,NaN,NaN
1,1,Tekening,"[""^TEKENING.*""]",NaN,NaN,NaN
2,2,Vondst,"[""^VONDSTENLIJST"", ""^VONDST$""]",NaN,NaN,NaN
3,3,Spoor,"[""^SPOREN$"", ""^SPOOR$""]",NaN,NaN,NaN
4,4,Vulling,"[""^VULLING.*"", ""MUUR""]",NaN,NaN,NaN
5,5,Dia,"[""DIA.*""]",NaN,NaN,NaN
6,6,Foto,"[""FOTO.*""]",NaN,NaN,NaN
7,7,Put,"[""^PUT$"", ""PUTTEN""]",NaN,NaN,NaN
8,8,Artefact,"[""ARTEFACT.*""]",NaN,NaN,NaN
9,9,Hout,"[""HOUT"", ""ARTF_OPH""]",Artefact,NaN,NaN


## Find all Attributes and connect them to the Objects

In [4]:

df = attributes.getAllAttributes()
df[df['Object'] == 'Artefact'].head(5)

,Object,Kolommen,omschrijvingen,count,projecten,Attribute,teller,percentage_gevuld
140,Artefact,(none),[],3,"[DC154, DC179, DC24_STADSKANTOOR]",,NaN,NaN
141,Artefact,ARTEFACT,[administratief nummer],7,"[DC154, DC179, DC008, DC24_STADSKANTOOR]",artefactnr,203.0,100.0
142,Artefact,BEGINDAT,[],2,"[DC179, DC24_STADSKANTOOR]",dateringvanaf,NaN,NaN
143,Artefact,BESCHRYF,[],2,"[DC179, DC24_STADSKANTOOR]",beschrijving,NaN,NaN
144,Artefact,COMPLEET,[],2,"[DC179, DC24_STADSKANTOOR]",compleetheid,NaN,NaN


In [5]:
lst = df['Object'].unique()
Objecten_lst = Dropdown(options = sorted(lst))

@interact(obj = Objecten_lst)
def print_city(obj):
    display(df[df.Object == obj].style.apply(lambda x: ["color: green" if bool(set(x.projecten) & set(['DC179', 'DC154'])) else "" for v in x], axis = 1))


interactive(children=(Dropdown(description='obj', options=('Aardewerk', 'Artefact', 'Bot', 'Dia', 'Foto', 'Gee…

In [6]:
def setNr(kolom):
    string = str(kolom)
    return string[:len(string)-6] + 'nr' if string.endswith('nummer') else string

df["sugg_kolom"] = df["Kolommen"].str.lower()
#df["sugg_kolom"] = df.apply(lambda x: setNr(df["sugg_kolom"]), axis=1)
df["sugg_omschr"] = df.apply(lambda x: x['omschrijvingen'][0].split()[0].lower() if len(x['omschrijvingen']) > 0 else "", axis=1)
df["sugg_omschr"] = df.apply(lambda x: setNr(x['sugg_omschr']), axis=1)
df["new_attr"] = ""
df["new_attr_artefact"] = ""
df.head(30)

,Object,Kolommen,omschrijvingen,count,projecten,Attribute,teller,percentage_gevuld,sugg_kolom,sugg_omschr,new_attr,new_attr_artefact
0,Aardewerk,(none),"[KEUZELIJST AARDEWERK 1 BAKSEL, KEUZELIJST AAR...",25,"[DC33, HHD2, DC27, DC20, DC37, DC24_STADSKANTO...",,NaN,NaN,(none),keuzelijst,,
1,Aardewerk,10a,[doosnummer],16,"[HHD2, DC33, DC27, DC20, DC37, DC21, DB34, DC0...",doosnr,6222.0,65.0,10a,doosnr,,
2,Aardewerk,10b,[tekeningnummer],16,"[HHD2, DC33, DC27, DC20, DC37, DB34, DC21, DC0...",tekeningnr,617.0,6.0,10b,tekeningnr,,
3,Aardewerk,10c,[dianummer],16,"[HHD2, DC33, DC27, DC20, DC37, DB34, DC21, DC0...",fotonr,327.0,3.0,10c,dianr,,
4,Aardewerk,10d,[digifotonummer],16,"[HHD2, DC33, DC27, DC20, DC37, DB34, DC21, DC0...",fotonr,329.0,3.0,10d,digifotonr,,
5,Aardewerk,11,"[determinatie, gedetermineerd door:]",16,"[HHD2, DC33, DC27, DC20, DC37, DC21, DB34, DC0...",,7079.0,74.0,11,determinatie,,
6,Aardewerk,12,[restauratie],16,"[HHD2, DC33, DC27, DC20, DC37, DC21, DB34, DC0...",restauratie,7567.0,79.0,12,restauratie,,
7,Aardewerk,13,"[exposabel? ja/nee, exposabel? ja/nee]",16,"[HHD2, DC33, DC27, DC20, DC37, DB34, DC21, DC0...",exposabel,7567.0,79.0,13,exposabel?,,
8,Aardewerk,1b,[publicatiecode],16,"[HHD2, DC33, DC27, DC20, DC37, DB34, DC21, DC0...",plek,5272.0,55.0,1b,publicatiecode,,
9,Aardewerk,1b,[publicatiecode],16,"[HHD2, DC33, DC27, DC20, DC37, DB34, DC21, DC0...",publicatiecode,5272.0,55.0,1b,publicatiecode,,


In [7]:
lst_objects = df['Object'].unique()

with pd.ExcelWriter(SUGGESTIE_XL) as writer:
    for obj in lst_objects:
        df[df.Object == obj].to_excel(writer, sheet_name=obj)
    writer.save()


In [21]:
lst = list(df_table[df_table.Overerven == 'Artefact']['Object'])
lst.sort()
#lst = lst.sort()
for tp in lst:
    print(tp + ' = \"' + tp + '\"')

Aardewerk = "Aardewerk"
Bot = "Bot"
Glas = "Glas"
Hoorn = "Hoorn"
Hout = "Hout"
Ivoor = "Ivoor"
Keramiek = "Keramiek"
Kleipijp = "Kleipijp"
Leer = "Leer"
Menselijk_Materiaal = "Menselijk_Materiaal"
Metaal = "Metaal"
Munt = "Munt"
Onbekend = "Onbekend"
Schelp = "Schelp"
Spijker = "Spijker"
Steen = "Steen"
Textiel = "Textiel"


In [29]:
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)

with engine.connect() as connection:
    df = pd.read_sql_query('SELECT brondata, artefactsoort from "Def_Artefact"', connection)

list(df['artefactsoort'].unique())

['Aardewerk',
 'Keramiek',
 'Spijker',
 'Menselijk_Materiaal',
 'Metaal',
 'Munt',
 'Kleipijp',
 'Steen',
 'Hout',
 'Glas',
 'Leer',
 'Bot']

In [30]:
lst

['Aardewerk',
 'Bot',
 'Glas',
 'Hoorn',
 'Hout',
 'Ivoor',
 'Keramiek',
 'Kleipijp',
 'Leer',
 'Menselijk_Materiaal',
 'Metaal',
 'Munt',
 'Onbekend',
 'Schelp',
 'Spijker',
 'Steen',
 'Textiel']